In [12]:
import pandas as pd
import os
import numpy as np

In [6]:
df = pd.read_excel(r'C:\Users\Пажилой биофизик\Downloads\Dataset (n = 1236).xlsx')
df.drop(columns='#', inplace=True)

# Table 1

**Average age**

In [15]:
f"{round(df['AGE'].mean(), 2)} ± {round(np.std(df['AGE']), 2)}"

'19.57 ± 2.13'

 # <font color='green'>Все ОК!</font>

**Female percent**

In [22]:
round((df['SEX (1=male)'].notnull().sum() - df['SEX (1=male)'].sum()) / df['SEX (1=male)'].notnull().sum()*100, 2)

71.93

 # <font color='green'>Все ОК!</font>

**Education status**

In [36]:
df.groupby('YEAR').agg(N=('YEAR', 'count'), Perc=('YEAR', lambda x: round(x.shape[0]/df.shape[0]*100, 1))).reset_index(drop=True)

,N,Perc
0,481,38.9
1,458,37.1
2,234,18.9
3,46,3.7
4,14,1.1
5,3,0.2


 # <font color='green'>Все ОК!</font>

**Faculty**

In [41]:
df.groupby('FAC').agg(N=('FAC', 'count'), Perc=('FAC', lambda x: round(x.shape[0]/df.shape[0]*100, 1))).reset_index(drop=True)

,N,Perc
0,59,4.8
1,79,6.4
2,67,5.4
3,16,1.3
4,52,4.2
5,100,8.1
6,276,22.3
7,9,0.7
8,185,15.0
9,205,16.6


 # <font color='green'>Все ОК!</font>

**University**

In [39]:
df.groupby('UN').agg(N=('UN', 'count'), Perc=('UN', lambda x: round(x.shape[0]/df.shape[0]*100, 1)))

,N,Perc
UN,,
РГППУ,377,30.5
УРФУ,139,11.2
ЧелГУ,520,42.1
ЮУРГУ,200,16.2


 # <font color='green'>Все ОК!</font>

# Table 2

In [137]:
columns = ['Extraversion', 
           'Agreeableness', 
           'Conscientiousness', 
           'Neuroticism', 
           'Openness to experiences', 
           'Somatic symptoms', 
           'Psychological distress']

In [49]:
d = {key: value for key, value in zip(['BF_Ext', 'BF_Agr', 'BF_Con','BF_Neu', 'BF_Ope','PH_Tot', 'DS_Tot'], columns)}
d

{'BF_Ext': 'Extraversion',
 'BF_Agr': 'Agreeableness',
 'BF_Con': 'Conscientiousness',
 'BF_Neu': 'Neuroticism',
 'BF_Ope': 'Openness to experiences',
 'PH_Tot': 'Somatic symptoms',
 'DS_Tot': 'Psychological distress'}

In [50]:
df.rename(d, inplace=True)

**M** and **SD**

In [59]:
df[columns].agg([np.mean, np.std]).T.apply(lambda x: round(x, 2))

,mean,std
Extraversion,7.28,2.01
Agreeableness,6.53,1.72
Conscientiousness,6.39,1.73
Neuroticism,6.48,2.08
Openness to experiences,7.50,1.84
Somatic symptoms,7.58,4.67
Psychological distress,20.48,12.32


 # <font color='green'>Все ОК!</font>

**cor**

In [81]:
cor = df[columns].corr().apply(lambda x: round(x, 2))
for i, col in enumerate(columns):
    cor.loc[col, columns[i:]] = np.nan
cor.drop(columns = columns[-1])

,Extraversion,Agreeableness,Conscientiousness,Neuroticism,Openness to experiences,Somatic symptoms
Extraversion,NaN,NaN,NaN,NaN,NaN,NaN
Agreeableness,0.12,NaN,NaN,NaN,NaN,NaN
Conscientiousness,0.22,0.10,NaN,NaN,NaN,NaN
Neuroticism,-0.16,-0.05,-0.10,NaN,NaN,NaN
Openness to experiences,0.13,0.02,0.10,0.05,NaN,NaN
Somatic symptoms,-0.13,-0.11,-0.15,0.30,0.01,NaN
Psychological distress,-0.30,-0.15,-0.23,0.48,0.01,0.62


# Psychological distress -- Neuroticism <font color='red'>не совпадает</font>

**pearson**

In [118]:
from scipy.stats import pearsonr
pearson = pd.DataFrame()
for col in df[columns]:
    pearson = pd.concat([pearson, df[columns].apply(lambda x: pearsonr(df[col], x)[1]).to_frame().rename(columns={0: col})], axis=1)
pearson.rename(columns={0: name for name in columns}, inplace=True)
for i, col in enumerate(columns):
    pearson.loc[col, columns[i:]] = np.nan
pearson.drop(columns = columns[-1]).apply(lambda x: np.where(x < 0.001, '*', round(x, 2)))
# * p<0.001

,Extraversion,Agreeableness,Conscientiousness,Neuroticism,Openness to experiences,Somatic symptoms
Extraversion,nan,nan,nan,nan,nan,nan
Agreeableness,*,nan,nan,nan,nan,nan
Conscientiousness,*,*,nan,nan,nan,nan
Neuroticism,*,0.1,*,nan,nan,nan
Openness to experiences,*,0.41,*,0.09,nan,nan
Somatic symptoms,*,*,*,*,0.63,nan
Psychological distress,*,*,*,*,0.72,*


 # <font color='green'>Все ОК!</font>

# Effects of sex and age

In [138]:
from scipy.stats import ttest_ind
columns.append('SEX (1=male)')
def t_test(col):
    return ttest_ind(df.loc[df['SEX (1=male)'] == 1, col], df.loc[df['SEX (1=male)'] == 0, col])

In [139]:
# Somatic symptoms
t_test('Somatic symptoms')

Ttest_indResult(statistic=-6.275156515747765, pvalue=4.825248828216943e-10)

In [141]:
# Psychological distress
t_test('Psychological distress')

Ttest_indResult(statistic=-4.662783380747374, pvalue=3.4588916341086303e-06)

**Какой тест был не понятно, но статистическая разница есть**

# Regression analysis

In [142]:
import statsmodels.api as sm

In [150]:
# Результаты со свободным членом
X = df[columns].drop(columns=['Psychological distress', 'SEX (1=male)'])
X = sm.add_constant(X) 
y = df['Psychological distress']
model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                              
==================================================================================
Dep. Variable:     Psychological distress   R-squared:                       0.525
Model:                                OLS   Adj. R-squared:                  0.522
Method:                     Least Squares   F-statistic:                     226.1
Date:                    Sun, 16 May 2021   Prob (F-statistic):          1.67e-194
Time:                            11:59:00   Log-Likelihood:                -4397.4
No. Observations:                    1236   AIC:                             8809.
Df Residuals:                        1229   BIC:                             8845.
Df Model:                               6                                         
Covariance Type:                nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                      11.7835      1.908      6.177      0.000       8.041      15.526
Extraversion               -1.0012      0.126     -7.925      0.000      -1.249      -0.753
Agreeableness              -0.3719      0.143     -2.608      0.009      -0.652      -0.092
Conscientiousness          -0.6028      0.145     -4.145      0.000      -0.888      -0.318
Neuroticism                 1.7790      0.124     14.398      0.000       1.537       2.021
Openness to experiences     0.1291      0.133      0.969      0.333      -0.132       0.390
Somatic symptoms            1.2888      0.055     23.376      0.000       1.181       1.397
==============================================================================
Omnibus:                       39.076   Durbin-Watson:                   1.898
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               43.457
Skew:                           0.406   Prob(JB):                     3.66e-10
Kurtosis:                       3.431   Cond. No.                         137.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [151]:
# Результаты без свободного члена
X = df[columns].drop(columns=['Psychological distress', 'SEX (1=male)'])
y = df['Psychological distress']
model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                   OLS Regression Results                                  
===========================================================================================
Dep. Variable:     Psychological distress   R-squared (uncentered):                   0.870
Model:                                OLS   Adj. R-squared (uncentered):              0.869
Method:                     Least Squares   F-statistic:                              1371.
Date:                    Sun, 16 May 2021   Prob (F-statistic):                        0.00
Time:                            12:00:32   Log-Likelihood:                         -4416.2
No. Observations:                    1236   AIC:                                      8844.
Df Residuals:                        1230   BIC:                                      8875.
Df Model:                               6                                                  
Covariance Type:                nonrobust                                                  
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Extraversion               -0.7260      0.120     -6.050      0.000      -0.961      -0.491
Agreeableness               0.0060      0.131      0.046      0.963      -0.250       0.262
Conscientiousness          -0.2684      0.137     -1.959      0.050      -0.537       0.000
Neuroticism                 2.0882      0.115     18.211      0.000       1.863       2.313
Openness to experiences     0.4523      0.124      3.639      0.000       0.208       0.696
Somatic symptoms            1.3625      0.055     24.939      0.000       1.255       1.470
==============================================================================
Omnibus:                       36.653   Durbin-Watson:                   1.878
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               40.115
Skew:                           0.398   Prob(JB):                     1.95e-09
Kurtosis:                       3.383   Cond. No.                         10.8
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Если строить со свободным членом (intercept), то $R^2$ становится близким к 0.5, что меньше, $R^2$ у модели без него. Есть ли смысл оставлять свободный член?
